In [1]:
import rdflib
from rdflib.plugins.sparql import prepareQuery
from tabulate import tabulate

In [2]:
filename = "../../data/04/ABox.ttl"

In [3]:
text1 = '''CQ_4.1
Return all `cultural resources` and both the beginning and the end time of the `time interval` of their `conception events`
'''

query1 = '''
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/> 
PREFIX frbroo: <http://iflastandards.info/ns/fr/frbr/frbroo/> 

SELECT ?resource ?beginning ?end
WHERE {
    ?conception_event a frbroo:F27_Work_Conception ;
                        frbroo:R16_initiated ?resource ;
                        crm:P4_has_time-span ?time_interval .
    ?time_interval crm:P82a_begin_of_the_begin ?beginning ;
                        crm:P82b_end_of_the_end ?end .
}
'''

In [4]:
text2 = '''CQ_4.2
Return the `agents` who carried out the `conception` of a `cultural resource` in a `time interval` that comprises "1590".
'''

query2 = '''
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/> 
PREFIX frbroo: <http://iflastandards.info/ns/fr/frbr/frbroo/> 
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#> 

SELECT ?agent ?beginning ?end
WHERE {
    ?conception_event a frbroo:F27_Work_Conception ;
                    frbroo:R16_initiated ?resource ;
                    crm:P14_carried_out_by ?agent ;
                    crm:P4_has_time-span ?time_interval .
    ?time_interval crm:P82a_begin_of_the_begin ?beginning ;
                        crm:P82b_end_of_the_end ?end .

    FILTER (xsd:dateTime(concat(str(?beginning), "T00:00:00")) < xsd:dateTime("1591-01-01T00:00:00") && 
    xsd:dateTime(concat(str(?end), "T00:00:00")) > xsd:dateTime("1589-12-31T00:00:00"))
}
'''

In [5]:
queries = [(text1, query1),
           (text2, query2)]

g = rdflib.ConjunctiveGraph()
g.parse(filename, format="turtle", encoding="utf-8")

for query in queries:
    q = prepareQuery(query[1])
    results = g.query(q)
    print(query[0])
    table = []
    for row in results:
        table.append([row[var] for var in results.vars])
    print(tabulate(table, headers=results.vars, tablefmt="psql"))

CQ_4.1
Return all `cultural resources` and both the beginning and the end time of the `time interval` of their `conception events`

+----------------------------------------------------+-------------+------------+
| resource                                           | beginning   | end        |
|----------------------------------------------------+-------------+------------|
| http://purl.org/changes/object/data/04/ALD-L1-work | 1589-01-01  | 1593-12-31 |
| http://purl.org/changes/object/data/04/ALD-1-work  | 1482-01-01  | 1482-12-31 |
| http://purl.org/changes/object/data/04/ALD-26-work | 1500-01-01  | 1599-12-31 |
+----------------------------------------------------+-------------+------------+
CQ_4.2
Return the `agents` who carried out the `conception` of a `cultural resource` in a `time interval` that comprises "1590".

+-----------------------------------------------------------+-------------+------------+
| agent                                                     | beginning   |